In [12]:
import pandas as pd

df = pd.read_csv(r"D:\Code\TFBOKRMAP\okrlist\rdtest111c.csv", usecols = ['Number','Title', 'statement']) 
df.head()

,Number,Title,statement
0,1000,Feel boring.,boolean
1,1100,Play,boolean
2,1110,Play Cards,boolean
3,1120,Play Basketball,boolean
4,1130,Play Video Game,boolean


In [17]:
import networkx as nx

G = nx.DiGraph(rankdir="LR")
#Setting Nodes and Node Attributes 
G.add_nodes_from(df['Number'])
print(G.nodes)
for index, row in df.iterrows():
    G.nodes[row['Number']]['Title'] = row['Title']
print(G.nodes(data=True))

[1000, 1100, 1110, 1120, 1130, 1131, 2000, 2100, 2110, 2120, 2130]
[(1000, {'Title': 'Feel boring.'}), (1100, {'Title': 'Play'}), (1110, {'Title': 'Play Cards'}), (1120, {'Title': 'Play Basketball'}), (1130, {'Title': 'Play Video Game'}), (1131, {'Title': 'Open Steam'}), (2000, {'Title': 'Feel Hungry'}), (2100, {'Title': 'Eat'}), (2110, {'Title': 'McDonalds'}), (2120, {'Title': 'Nightmarket'}), (2130, {'Title': 'Dirt'})]


In [ ]:
#Setting Edges and Edge Attributes
#Build Edge List
#import pprint
#edge_list=[]
#for i in range(1000,5901):
#  a=i//1000
#  b=(i%1000)//100
#  c=(i%100)//10
#  d=i%10
#  if b!=0:
#    if c!=0:
#      edge_list.append(i)
#    else:
#      if d==0:
#        edge_list.append(i)
#  else:
#    if c==0 and d==0:
#      edge_list.append(i)
#pprint.pprint(edge_list, depth=1, width=10000)
#這段好像沒有用.....87

In [29]:
#Build Edge List2
import itertools
import csv

#讀取參照的edge結構
with open('number_logic.csv', newline='') as f:
    reader = csv.reader(f)
    number_logic = [tuple(map(int, row)) for row in reader]
#print(number_logic)

edge_list=[] #把df裡面的節點抓成list
for i in df['Number']:
    edge_list.append(i)
#print(edge_list)

edge_raw = [] #把df裡面的節點組合成edge tuple的list
for i in itertools.combinations(edge_list, 2):
      edge_raw.append(i)
#print(edge_raw)

#比對edge_raw跟number_logic然後挑選集合部分做出新list
edge = [x for x in edge_raw if x in number_logic]
print(edge)
G.add_edges_from(edge)

#設計Edge Labels
statement = df['statement'].tolist()
edge_label = {}

for k, v in zip(G.edges, statement):
    edge_label[k] = v
#print(edge_label)

[(1000, 1100), (1100, 1110), (1100, 1120), (1100, 1130), (1130, 1131), (2000, 2100), (2100, 2110), (2100, 2120), (2100, 2130)]
{(1000, 1100): 'boolean', (1100, 1110): 'boolean', (1100, 1120): 'boolean', (1100, 1130): 'boolean', (1130, 1131): 'boolean', (2000, 2100): 'boolean', (2100, 2110): 'boolean', (2100, 2120): 'boolean', (2100, 2130): 'boolean'}


In [ ]:
#使用Graphviz繪圖
